# ⭐ Tutorial: The Power of CPCV (`CombinatorialPurged`)

In the previous tutorial, we used `PurgedKFold` to get a single, reliable out-of-sample (OOS) accuracy score. But this leads to a critical question:

**"How do we know we weren't just lucky?"**

A single backtest path, even a purged one, could be an outlier. `CombinatorialPurged` (CPCV) solves this. Instead of one path, it runs *many* backtest paths by combining different train/test splits. This allows us to see the **distribution** of performance metrics, giving a much more robust assessment of our strategy.

This notebook will show you how to:
1.  Run `CombinatorialPurged` to get *all* backtest paths.
2.  Calculate the performance (e.g., accuracy) for *each* path.
3.  Plot a histogram of these performance scores to see the mean, variance, and full range of outcomes.

## 0. Setup and Imports

This setup is identical to the `PurgedKFold` tutorial for consistency.

In [5]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings

# Third-party for data and modeling
import yfinance as yf
import ta
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# RiskLabAI Imports
from RiskLabAI.backtest.validation import CrossValidatorController
import RiskLabAI.utils.publication_plots as pub_plots

# Setup plotting and configuration
pub_plots.setup_publication_style()
warnings.filterwarnings('ignore')

AttributeError: module 'numba' has no attribute 'core'

## 1. Load Data & Generate Signals

We use the same data generation as the previous tutorial to create our `X`, `y`, and `event_times`.

In [ ]:
# --- 1. Load Data ---
# Use the stable yf.Ticker().history() method.
# We use "SPY" (the ETF) and auto_adjust=True.
# This correctly gives us the dividend-adjusted 'Close' price.
ticker_symbol = "SPY"
start_date = "2010-01-01"
end_date = "2025-01-01"

data = yf.Ticker(ticker_symbol).history(
    start=start_date, 
    end=end_date, 
    auto_adjust=True  # Automatically adjusts for dividends/splits
)

# --- 2. Generate Features (X) ---
X = pd.DataFrame(index=data.index)
# data['Close'] is the dividend-adjusted price
X['rsi'] = ta.momentum.RSIIndicator(data['Close'], window=14).rsi()
X['roc'] = ta.momentum.ROCIndicator(data['Close'], window=10).roc()

# --- 3. Generate Labels (y) ---
look_forward = 20
y = data['Close'].pct_change(look_forward).shift(-look_forward)
y = pd.Series((y > 0).astype(int), name='label')

# --- 4. Define Event Times (t1) ---
# The label's 'end time' is 20 days after the observation.
event_times = pd.Series(y.index, index=y.index).apply(lambda x: x + pd.DateOffset(days=look_forward))

# --- 5. Align Data ---
# Combine all data and drop NaNs to ensure X, y, and t1 are aligned
all_data = pd.concat([X, y, event_times.rename('t1')], axis=1).dropna()

X = all_data.drop(columns=['label', 't1'])
y = all_data['label']
event_times = all_data['t1']

print(f"--- SPY (ETF) Data Loaded ---")
print(f"Aligned data shapes: X={X.shape}, y={y.shape}, t1={event_times.shape}")
print(X.head())

[*********************100%***********************]  1 of 1 completed


KeyError: 'Close'

## 2. Running the CPCV Backtest

This is the core of the tutorial. Instead of manually looping, we will use the `backtest_predictions` method built into our cross-validators. This method is highly optimized and uses parallel processing.

We define:
* `n_splits = 10`: The data is split into 10 groups.
* `n_test_groups = 2`: Each test set will consist of 2 of these groups.

This will generate $C(10, 2) = 45$ total splits and, from those, $k=5$ backtest paths. (Note: $k$ is $n\_splits / n\_test\_groups$).


In [ ]:
n_splits = 10
n_test_groups = 2
embargo_pct = 0.01

# --- 1. Instantiate the Validator ---
# We use the CrossValidatorController for easy creation.
cv_controller = CrossValidatorController(
    validator_type='combinatorialpurged',
    n_splits=n_splits,
    n_test_groups=n_test_groups,
    times=event_times,
    embargo=embargo_pct
)

cv = cv_controller.get_validator()

# --- 2. Define the Model ---
model = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)

# --- 3. Run All Backtest Paths --- 
print(f"Running Combinatorial Backtest... (This may take a moment)")

# This single call runs all C(n,k) training jobs and assembles all paths.
# n_jobs=-1 will parallelize the training of the 45 estimators.
paths_predictions = cv.backtest_predictions(
    estimator=model,
    data=X,
    labels=y,
    n_jobs=-1
)

print(f"Backtest complete.")
print(f"Received {len(paths_predictions)} backtest paths.")
print(f"Keys: {list(paths_predictions.keys())}")

## 3. Evaluate the Distribution of Performance

This is the "Aha!" moment. `paths_predictions` is not a single array, but a **dictionary** where each key is a path ID and each value is the OOS prediction array for that path.

We can now calculate the accuracy for *each path* and see the full distribution of our model's performance.

In [ ]:
path_accuracies = []

print("Calculating accuracy for each path...")
for path_name, predictions in paths_predictions.items():
    # Align true labels with this path's predictions
    # Note: All paths have the same length and indices in this implementation
    y_true = y.loc[predictions.index]
    
    # Calculate accuracy
    acc = accuracy_score(y_true, predictions)
    path_accuracies.append(acc)
    print(f"  {path_name}: Accuracy = {acc * 100:.2f}%")

path_accuracies = pd.Series(path_accuracies)


## 4. Visualize the Results

Now we plot the histogram of our path accuracies. This tells us far more than a single score.

In [ ]:
mean_acc = path_accuracies.mean()
std_acc = path_accuracies.std()
p5 = path_accuracies.quantile(0.05)
p95 = path_accuracies.quantile(0.95)

fig, ax = plt.subplots(figsize=(10, 6))

ax.hist(path_accuracies, bins=20, alpha=0.7, color='#00aedb', edgecolor='black')

ax.axvline(mean_acc, color='#d11141', linestyle='--', lw=2, 
             label=f'Mean Acc: {mean_acc*100:.2f}%')
             
ax.axvspan(p5, p95, color='#f37735', alpha=0.2, 
             label=f'90% C.I. [{p5*100:.2f}% - {p95*100:.2f}%]')

ax.set_xlabel('Out-of-Sample Accuracy')
ax.set_ylabel('Frequency (Number of Paths)')
ax.set_title(f'Distribution of CPCV Path Accuracies (k={len(path_accuracies)} paths)')
ax.legend()
plt.tight_layout()
plt.show()

print(f"Summary Statistics:")
print(f"Mean Accuracy: {mean_acc*100:.2f}%")
print(f"Std. Dev:      {std_acc*100:.2f}%")
print(f"Min Accuracy:  {path_accuracies.min()*100:.2f}%")
print(f"Max Accuracy:  {path_accuracies.max()*100:.2f}%")

## Conclusion

We have successfully moved from a single, potentially misleading, performance score to a full-blown *distribution* of performance.

We now have a much more honest and robust understanding of our strategy. Instead of just saying "the accuracy is 5X.XX%," we can now say:

"The strategy's mean accuracy is **$Y\%$**, with a 90% confidence interval of **$[A\%, B\%]$**. In the worst-case paths, performance dropped to **$Min\%$**, and in the best-case, it rose to **$Max\%$**."

This result, which accounts for the stochasticity of the train/test split, is a core requirement for robust financial machine learning.